In [16]:
import gzip
from tqdm import tqdm
second_3 = []
second_jump = []
with gzip.open('second_3.gz','rb') as f:
    for line in tqdm(f,desc = "Processing lines",unit = "line"):
        triplet = line.strip().decode().split(" ")[:3]
        second_3.append(triplet)
with gzip.open('second_entity.gz','rb') as f:
    for line in tqdm(f,desc = "Processing lines",unit = "line"):
        second_jump.append(line.strip().decode())

print(len(second_3))
print(len(second_jump))
print(second_3[4])
print(second_jump[4])

Processing lines: 9129030line [00:17, 508571.94line/s]
Processing lines: 9129030line [00:08, 1036078.38line/s]

9129030
9129030
['<http://rdf.freebase.com/ns/award.award_winner>', '<http://rdf.freebase.com/ns/type.type.instance>', '<http://rdf.freebase.com/ns/m.0534v>']
<http://rdf.freebase.com/ns/award.award_winner>


In [24]:
print(second_3[5][1])

<http://rdf.freebase.com/ns/type.type.instance>


In [30]:
import copy
min_relation = 50
min_entity = 15
max_entity = 20000

#函数，通过关系小于50更新second_3，通过second_3更新second_jump
def update_second_3_by_relation ():
    num_relation = {}#存储每个关系的数量
    num_of_entity = len(second_3)#实体的数量
    for i in range(num_of_entity):
        if second_3[i][1] not in num_relation:
            num_relation[second_3[i][1]] = 1
        else:
            num_relation[second_3[i][1]] = num_relation[second_3[i][1]] + 1
    id_to_delete = []#存储需要删除的三元组的id
    #在second_3中删除关系小于50的三元组,并在second_jump中删除对应的实体
    for i in range(num_of_entity):
        if num_relation[second_3[i][1]] <= min_relation:
            id_to_delete.append(i)
    for i in range(len(id_to_delete)):
        del second_3[id_to_delete[i]-i]
        del second_jump[id_to_delete[i]-i]
    
def update_second_3_by_entity():
    num_entity = {}#存储每个实体的数量
    num_of_entity = len(second_3)#实体的数量,即三元组的数量
    for i in range(num_of_entity):
        if second_jump[i] not in num_entity:
            num_entity[second_jump[i]] = 1
        else:
            num_entity[second_jump[i]] = num_entity[second_jump[i]] + 1
    id_to_delete = []#存储需要删除的三元组的id
    #在second_3中删除实体小于20且不在entity_id_list的三元组,并在second_jump中删除对应的实体
    for i in range(num_of_entity):
        if num_entity[second_jump[i]] <= min_entity or num_entity[second_jump[i]] >= max_entity:
            id_to_delete.append(i)
    for i in range(len(id_to_delete)):
        del second_3[id_to_delete[i]-i]
        del second_jump[id_to_delete[i]-i]
            
#处理第二跳，删除关系小于50的三元组，删除实体小于20的三元组,轮流执行直至second_3和second_jump不再变化
def process_second_jump():
    second_3_old = copy.deepcopy(second_3)
    second_jump_old = copy.deepcopy(second_jump)
    update_second_3_by_entity()
    update_second_3_by_relation()
    while second_3_old != second_3 and second_jump_old != second_jump:
        second_3_old = copy.deepcopy(second_3)
        second_jump_old = copy.deepcopy(second_jump)
        update_second_3_by_entity()
        update_second_3_by_relation()
    
process_second_jump()
    

In [27]:
print(len(second_3))
print(len(second_jump))



9120623
9120623


In [28]:
#测试代码
#测试正确性
num_entity_second_jump = {}
num_relation_second_jump = {}
num_of_entity = len(second_jump)
for i in range(num_of_entity):
    if second_jump[i] not in num_entity_second_jump:
        num_entity_second_jump[second_jump[i]] = 1
    else:
        num_entity_second_jump[second_jump[i]] = num_entity_second_jump[second_jump[i]] + 1
    if second_3[i][1] not in num_relation_second_jump:
        num_relation_second_jump[second_3[i][1]] = 1
    else:
        num_relation_second_jump[second_3[i][1]] = num_relation_second_jump[second_3[i][1]] + 1

#测试正确性,测试数量是否符合要求
min_relation = 50
min_entity = 15
for i in range(num_of_entity):
    if num_entity_second_jump[second_jump[i]] <= min_entity:
        print("error")
        print(i)
        print(second_jump[i])
    if num_relation_second_jump[second_3[i][1]] <= min_relation:
        print("error")
        print(i)
        print(second_3[i][1])

print(len(num_entity_second_jump))
print(len(num_relation_second_jump))

389668
133
